In [576]:
%load_ext autoreload
%autoreload 2

from schema2db import validate
from schema2db.parse_schema import SchemaParser
from schema2db.randomdata import *
from schema2db.gendata import DBGenerator
import pandas as pd
import numpy as np
import re

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [531]:
random_list('varchar', [20], 10)

['qbu',
 'zmmdwuaavpchejrzkkj',
 'fzatkdcvncvd',
 'xaod dyeks',
 'jrldqus',
 'ziob',
 'ingtjjczcrgpr',
 ' extn y rgjexyy',
 'qywaeevqmyfvcqe',
 'bmekigq sfwlzuvljk',
 'iu yquduqojei',
 'hysbxdilev',
 'izcmk',
 'nhcwu tzkny egy',
 'pxekgosfagiziht',
 'oisyaoskktqt',
 'skunebqm uz  csht',
 'fqyl gbpysbekavi',
 'zuddfr',
 'kmthqryczcuxxpgw',
 'lhcgialjmapyp',
 'tnpxegqjousgpexpyvk',
 'ichfjsqnmklcmyervd',
 'lonwyahggvrp',
 'efouzzl',
 'gcijl fqjcpcr',
 'wi',
 ' rnymmcrbjgiohsdnive',
 'vfj pwc a cckz',
 'povenvw ',
 'johdlzhlymlzzuc j',
 'sinqyk',
 'q',
 'zyppoawcbwp',
 'uqavqxvberyaanmfi',
 'qyzo aaitlqb',
 ' wfwuzmcnc',
 'ygqfkbkjx',
 'nirvmytfurtofwiegs',
 'tcgmx',
 'pkrtynfgkeoerk',
 'guzrx',
 'fytpiepcutzaltin',
 'pnsmgrwau',
 'zpb wdf',
 'x',
 'fhpelynh',
 'gnpeigawefurwaahzo',
 'vsxqgpbkk',
 'b']

In [532]:
print(random_int(signed=False))
print(random_varchar())
print(random_decimal(4,2))

7155
rgm
-52.53


In [533]:
inputfile = '../tests/testdata/testschema1.sql'
with open(inputfile, 'r') as f:
    inputstr = f.read()

In [534]:
parser = SchemaParser()
parsed = parser.extract_sql_doc("../tests/testdata/testschema1.sql")
for p in parsed:
    print(p)
    print(parsed[p])
    print('---')

create
{'prices': {'columns': [{'name': 'itemid', 'type': {'type': 'int', 'args': ['16'], 'signed': False}, 'null': False}, {'name': 'itemname', 'type': {'type': 'varchar', 'args': ['20']}, 'null': False}, {'name': 'price', 'type': {'type': 'decimal', 'args': ['9', '6'], 'signed': True}, 'null': False}, {'name': 'sold', 'type': {'type': 'int', 'args': ['16'], 'signed': False}, 'null': False, 'default': '0,'}], 'primary_keys': [['itemid']], 'tablename': 'prices', 'operation': 'create'}, 'sales': {'columns': [{'name': 'saleid', 'type': {'type': 'int', 'args': ['16'], 'signed': False}, 'null': False}, {'name': 'itemid', 'type': {'type': 'int', 'args': ['16'], 'signed': False}, 'null': False}, {'name': 'saledate', 'type': {'type': 'date', 'args': []}, 'null': False}, {'name': 'quantity', 'type': {'type': 'int', 'args': ['16'], 'signed': False}, 'null': False}, {'name': 'buyername', 'type': {'type': 'varchar', 'args': ['60']}, 'null': True}], 'primary_keys': [['saleid']], 'tablename': 'sale

In [587]:
create = [p for p in parsed.get('create')]
alter = [p for p in parsed.get('alter')]
waiting_room = create
print(create)
print("----")
print(alter)

processed = []
db = {}

while len(waiting_room) > 0:
    start_count = len(waiting_room)
    end_count = start_count
    for i in range(start_count):
        cand = waiting_room.pop()
        valid = True
        foreign_tables = [ref['referenced'] for ref in parsed.get('alter').get(cand, {}).get('foreign_keys', [])]
        for f in foreign_tables:
            if f not in processed:
                waiting_room.insert(0, cand)
                valid = False
                break
        if valid:
            table_args = parsed['create'][cand]
            constraints = parsed.get('alter').get(cand, {})
            db[cand] = gen_table(cand, table_args, constraints)
            processed.append(cand)
            end_count -= 1
    
    if start_count == end_count:
        raise ValueError("There are some circular dependencies in foreign keys. Please double check your constraints")

['prices', 'sales']
----
['sales', 'prices']
waiting: ['prices', 'sales']
processing prices
{'check': [{'type': 'enum', 'column': 'sold', 'values': ['0', '1']}], 'foreign_keys': [], 'tablename': 'prices', 'operation': 'alter'}
itemid
{'datatype': 'int', 'args': [16], 'signed': False, 'isnull': False, 'primary_key': False}
{'datatype': 'int', 'args': [16], 'signed': False, 'isnull': False, 'primary_key': False}
itemname
{'datatype': 'varchar', 'args': [20], 'signed': None, 'isnull': False, 'primary_key': False}
{'datatype': 'varchar', 'args': [20], 'signed': None, 'isnull': False, 'primary_key': False}
price
{'datatype': 'decimal', 'args': [9, 6], 'signed': True, 'isnull': False, 'primary_key': False}
{'datatype': 'decimal', 'args': [9, 6], 'signed': True, 'isnull': False, 'primary_key': False}
sold
{'datatype': 'int', 'args': [16], 'signed': False, 'isnull': False, 'primary_key': False}
['0', '1']
{'datatype': 'int', 'args': [16], 'signed': False, 'isnull': False, 'primary_key': False,

In [572]:
#dbgenerator.gen_column_data(datatype='varchar', args=[5], signed=True)

In [586]:
dbgenerator = DBGenerator(parsed)

def gen_table(tablename, create_sql, constrain_sql, row_num=50):
    enums = {}
    foreign_keys = {}
    kwargs = {}
    rows = row_num
    table = None
    for c in constrain_sql.get('check', []):
        if c['type'] == 'enum':
            if c['column'] in enums:
                enums[c['column']].append(c)
            else:
                enums[c['column']] = [c]
    for c in constrain_sql.get('foreign_keys', []):
        if c['column'] in foreign_keys:
            foreign_keys[c['column']].append(c)
        else:
            foreign_keys[c['column']] = [c]
    for col_sql in create_sql['columns']:
        name = col_sql['name']
        kwargs['datatype'] = col_sql['type']['type']
        kwargs['args'] = [int(arg) for arg in col_sql['type']['args']]
        kwargs['signed'] = col_sql['type'].get('signed')
        kwargs['isnull'] = col_sql.get('null')
        kwargs['primary_key'] = name in create_sql['primary_keys']

        if enums.get(name):
            choices = enums.get(name)[0]['values']
            kwargs['choices'] = choices
            rows = min(len(choices), rows)
        elif foreign_keys.get(name):
            d = foreign_keys.get(name)[0]
            choices = db[d['referenced']][d['source_column']].tolist()
            kwargs['choices'] = choices
            rows = min(len(choices), rows)
        rowdata = dbgenerator.gen_column_data(**kwargs)
        if table is None:
            table = pd.DataFrame({name: rowdata})
        else:
            table[name] = rowdata
    return table
            

In [588]:
db

{'prices':     itemid              itemname       price sold
 0      502                  qxre  672.793110    1
 1     9852                     x   69.232564    0
 2      489              ietimmkw -128.791352    0
 3     2100     ddbppnlszodcnl fu -297.874681    0
 4     2712            jyrgsnrvnf  672.490051    1
 5     7138                 ekjdj  511.694945    0
 6     4025              uxswvpwh  270.800763    1
 7     1489                    zl -242.482345    0
 8     1309    tkagtnqhsamhzfrfe  -293.186089    0
 9     9814                 qknmm  798.600793    1
 10    3882                 qcd d -967.478161    0
 11    3549       pcxhrknredffquq  439.911487    0
 12    4284            rwyqppuuyh  209.804672    0
 13    8699     ojpzpvgtvlmev srt  316.114771    1
 14     340     edpwlbmnfrmfwnbcg  385.482890    0
 15    9476            jdujruqh t -106.448668    0
 16    5266                     k -164.265570    1
 17    1804       evrqfgkdokernum  713.361260    1
 18     270  dnrepwkv

In [450]:
parsed['create']['sales']

{'columns': [{'name': 'saleid',
   'type': {'type': 'int', 'args': ['16'], 'signed': False},
   'null': False},
  {'name': 'itemid',
   'type': {'type': 'int', 'args': ['16'], 'signed': False},
   'null': False},
  {'name': 'saledate', 'type': {'type': 'date', 'args': []}, 'null': False},
  {'name': 'quantity',
   'type': {'type': 'int', 'args': ['16'], 'signed': False},
   'null': False},
  {'name': 'buyername',
   'type': {'type': 'varchar', 'args': ['60']},
   'null': True}],
 'primary_keys': [['saleid']],
 'tablename': 'sales',
 'operation': 'create'}

In [442]:
for i in parsed.get('alter').get('sales',{}).get('foreign_keys', []):
    print(i)

{'from': 'itemid', 'to': 'itemid', 'referenced': 'prices'}


In [215]:
def clean_doc(doc):
    """clean out extra spaces
    """
    lines = doc.split('\n')
    lines = [" ".join(l.split()) for l in lines if l and not l.isspace()]
    return '\n'.join(lines)

def get_table_name(line):
    words = line.split()
    if words[1].lower() != 'table'.lower():
        raise ValueError("You can only create tables, not {}".format(words[1]))
    else:
        return words[2]

def extract_sql_doc(inputfile):
    """ Extract sql components from the entire document
    params:
    inputfile: str, name of the input sql file
    """
    with open(inputfile, 'r') as f:
        inputstr = f.read()
    blocks = clean_doc(inputstr).split(";\n")
    components = []
    for b in blocks:
        components.append(extract_sql_block(b))
    return components

def extract_sql_block(block):
    """ Parse a block of sql commands and return corresponding
    sql components
    """
    operation = block.split()[0].lower()
    if operation == 'create':
        return parse_create_block(block)
    elif operation == 'alter':
        return parse_alter_block(block)
    else:
        raise NotImplementedError("{} operation not supported".format(operation))

def extract_datatype(words):
    strtype = {}
    next_pos = 1
    strtype['type'] = (re.findall("[a-zA-Z]+", words[0]))[0].lower()
    strtype['args'] = re.findall("[0-9]+", words[0])
    if len(words) > 1 and words[1].lower() == 'signed':
        strtype['signed'] = True
        next_pos += 1
    elif len(words) > 1 and words[1].lower() == 'unsigned':
        strtype['signed'] = False
        next_pos += 1
    return strtype, words[next_pos:]

def extract_null(words):
    ifnull = None
    nextpos = 0
    if 'not null' in " ".join([s.lower() for s in words]):
        ifnull = False
        nextpos = 2
    elif 'null' in " ".join([s.lower() for s in words]):
        ifnull = True
        nextpos = 1
    if len(words) > nextpos:
        return ifnull, words[1:]
    else:
        return ifnull, None

def parse_create_block(sql_str):
    lines = sql_str.split('\n')
    table = {'columns': [], 'primary_keys': []}
    table['tablename'] = get_table_name(lines[0])
    for l in lines[1:]:
        if 'primary key' in l.lower():
            table['primary_keys'].append(parse_keys(l))
        elif re.findall("[a-zA-Z]+", l):
            table['columns'].append(parse_items(l))
    return table

def parse_items(l):
    c = l.split()
    cmds = []
    cmds = {'name': c[0]}
    cmds['type'], cmd_remaining = extract_datatype(c[1:])
    cmds['null'], cmd_remaining = extract_null(cmd_remaining)
    return cmds

def parse_keys(l):
    tokens = re.findall("[a-zA-Z]+", l)
    return [t for t in tokens if t.lower() not in('primary', 'key')]

def parse_alter_block(sql_cmds):
    constraints = ["constraints placeholder"]
    return constraints

In [216]:
extract_sql_doc('../tests/testdata/testschema1.sql')

[{'columns': [{'name': 'itemid',
    'type': {'type': 'int', 'args': ['16'], 'signed': False},
    'null': False},
   {'name': 'itemname',
    'type': {'type': 'varchar', 'args': ['20']},
    'null': False},
   {'name': 'price',
    'type': {'type': 'decimal', 'args': ['9', '6'], 'signed': True},
    'null': False}],
  'primary_keys': [['itemid']],
  'tablename': 'prices'},
 {'columns': [{'name': 'saleid',
    'type': {'type': 'int', 'args': ['16'], 'signed': False},
    'null': False},
   {'name': 'itemid',
    'type': {'type': 'int', 'args': ['16'], 'signed': False},
    'null': False},
   {'name': 'saledate', 'type': {'type': 'date', 'args': []}, 'null': False},
   {'name': 'quantity',
    'type': {'type': 'int', 'args': ['16'], 'signed': False},
    'null': False},
   {'name': 'buyername',
    'type': {'type': 'varchar', 'args': ['60']},
    'null': True}],
  'primary_keys': [['saleid']],
  'tablename': 'sales'},
 ['constraints placeholder']]

In [208]:
re.findall("[a-zA-Z]+", "primary   key (one, two, three)")

['primary', 'key', 'one', 'two', 'three']

In [233]:
['ok', 'fine'].index('okk')

ValueError: 'okk' is not in list

In [91]:
parse_create(table_words[3])

{'colname': 'price',
 'type': {'type': 'decimal', 'args': ['9', '6'], 'signed': True},
 'null': False}

In [50]:
" ".join(" not  null".split())

'not null'

In [242]:
alterblock = """alter table sales
   add constraint fk_sales_itemid_prices_itemid foreign key (itemid) references prices (itemid)
   ADD CONSTRAINT chk_sales_sold CHECK (sold IN (0, 1));"""

In [243]:
alterblock

'alter table sales\n   add constraint fk_sales_itemid_prices_itemid foreign key (itemid) references prices (itemid)\n   ADD CONSTRAINT chk_sales_sold CHECK (sold IN (0, 1));'

In [245]:
lines = alterblock.split('\n')

In [247]:
tablename = get_table_name(lines[0])
tablename

'sales'

In [249]:
lines[1]

'   add constraint fk_sales_itemid_prices_itemid foreign key (itemid) references prices (itemid)'

In [257]:
tokens = lines[1].split()
tokens = [_clean_token(t) for t in tokens]

In [256]:
def _clean_token(s):
    return re.sub(r'\W+', '', s)

In [253]:
re.sub(r'\W+', '', '(item2)')

'item2'

In [258]:
tokens

['add',
 'constraint',
 'fk_sales_itemid_prices_itemid',
 'foreign',
 'key',
 'itemid',
 'references',
 'prices',
 'itemid']

In [ ]:
reference = {''}

In [259]:
if tokens[2][:2].lower() == 'fk':
     parsed = parse_foreign_keys(tokens)

SyntaxError: 'return' outside function (<ipython-input-259-d2bc60ce054d>, line 2)

In [289]:
['1', '2'].index('1')

0